In [49]:
import torch
import numpy as np
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import math
import pandas as pd
from prophet import Prophet
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype


In [50]:
class random_baseline_model():
    def __init__(self, filename, sensor):
        self.sensor = sensor
        self.data = pd.read_csv(filename,
                        usecols=[0,sensor + 1],
                        index_col=[0],
                        parse_dates=[0])
        
        label = str(sensor)
        cat_type = CategoricalDtype(categories=['Monday','Tuesday',
                                            'Wednesday',
                                            'Thursday','Friday',
                                            'Saturday','Sunday'],
                                ordered=True)

        df = self.data.copy()

        # here there are a few more features that are not utilized by this model for simplicity
        # but can be added later 
        df['date'] = df.index
        df['hour'] = df['date'].dt.hour
        df['dayofweek'] = df['date'].dt.dayofweek
        df['weekday'] = df['date'].dt.day_name()
        df['weekday'] = df['weekday'].astype(cat_type)
        df['quarter'] = df['date'].dt.quarter
        df['month'] = df['date'].dt.month
        df['year'] = df['date'].dt.year
        df['dayofyear'] = df['date'].dt.dayofyear
        df['dayofmonth'] = df['date'].dt.day
        df['date_offset'] = (df.date.dt.month*100 + df.date.dt.day - 320)%1300

        df['season'] = pd.cut(df['date_offset'], [0, 300, 602, 900, 1300], 
                            labels=['Spring', 'Summer', 'Fall', 'Winter']
                    )

        # Other features are not that useful 
        X = df[['hour', 'weekday','season']]
        y = df[label]

        self.features_and_target =  pd.concat([X, y], axis=1)
        self.group = self.features_and_target.groupby(['hour', 'weekday', 'season'])[str(sensor)].mean()
        self.pivot_group = self.features_and_target.groupby(['hour', 'weekday', 'season'])[str(sensor)].mean().reset_index()

    
    # Time is pd.DatetimeIndex()
    # example: time  = pd.DatetimeIndex(["8/1/2016 10:00:00"])
    # returns a numpy.float64 which is the prediction for the date provided
    def predict_one(self, time):
        date_offset = (time.month*100 + time.day- 320)%1300

        season = pd.cut(date_offset, [0, 300, 602, 900, 1300], 
                            labels=['Spring', 'Summer', 'Fall', 'Winter'])
        return self.group[time.hour[0]][time.day_name()[0]][season[0]]
    
    

    # format of the file: date - value
    # returns a numpy.ndarray of the predicted values only (numpy.float64)
    def predict_many(self, filename):
        dates = pd.read_csv(filename, usecols=[0],
                        index_col=[0],
                        parse_dates=[0])

        output = np.zeros(len(dates.index))
       
        date_offset = (time.month*100 + time.day- 320)%1300
        season = pd.cut(date_offset, [0, 300, 602, 900, 1300], 
                            labels=['Spring', 'Summer', 'Fall', 'Winter'])
        i = 0
        for date in dates.index:            
            output[i] = (self.group[pd.DatetimeIndex([date]).hour[0]][pd.DatetimeIndex([date]).day_name()[0]][season[0]])
            i += 1

        return output

    # doesn't work well right now lmao
    # index : array of strings: either one or two of ["hour", "weekday", "season"]
    # string one of "hour", "weekday", "season"
    def plot(self, index, column):
        pivot_table = self.pivot_group.pivot_table(index=index, columns=column, values=str(self.sensor))
        pivot_table.plot(kind='bar', figsize=(10, 6))
        plt.title(f'Mean Target Value by Hour, Day, and Season')
        plt.xlabel(index)
        plt.ylabel('Mean Target Value')
        plt.xticks(rotation=45)
        plt.legend(title=column, bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()


In [51]:
sensor = 6
model = random_baseline_model('./sample-data/traffic.csv', sensor)

C:\Users\user\AppData\Local\Temp\ipykernel_6644\3104805867.py:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self.group = self.features_and_target.groupby(['hour', 'weekday', 'season'])[str(sensor)].mean()
C:\Users\user\AppData\Local\Temp\ipykernel_6644\3104805867.py:42: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self.pivot_group = self.features_and_target.groupby(['hour', 'weekday', 'season'])[str(sensor)].mean().reset_index()


In [52]:
time = pd.DatetimeIndex(["11/13/2016 10:00:00"])
k = model.predict_one(time)
k

0.05936923076923077

In [53]:
# predict the test data
filename = "sensor_" + str(sensor) + ".csv"
print(filename)
predictions = model.predict_many(filename)

sensor_6.csv


In [54]:
# get the actual values from the test data
actual_values_df = pd.read_csv("sensor_" + str(sensor) +".csv", usecols=[1],
                        index_col=[0],
                        parse_dates=[0])

actual_values = np.zeros(len(actual_values_df.index))
for i in range(len(actual_values_df.index)):
    actual_values[i] = (actual_values_df.index[i])

C:\Users\user\AppData\Local\Temp\ipykernel_6644\1555280279.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  actual_values_df = pd.read_csv("sensor_" + str(sensor) +".csv", usecols=[1],


In [55]:
# get a file for the test data with the dates only
df = pd.read_csv("sensor_"+ str(sensor)+ "_dates_only.csv")

# add the model's predictions to that file
df[sensor] = predictions
df.to_csv("sensor_" + str(sensor) +"_predition.csv", index=False)

In [56]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
epsilon = 0.00000001
r2 = r2_score(actual_values, predictions)
mae = mean_absolute_error(actual_values, predictions)
mse = mean_squared_error(actual_values, predictions)

print(f"r2 = {r2}")
print(f"mae = {mae}")
print(f"mse = {mse}")

# for sensor 6:
# r2 = 0.8529282859104332
# mae = 0.013322875828815445
# mse = 0.0006458734260370798

r2 = 0.8264651917664867
mae = 0.01295616445997569
mse = 0.0007620875422870853
